In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import h5py
from selenium import webdriver
import time

In [2]:
base_dir="https://magic.wizards.com/"
base_url="https://magic.wizards.com/en/content/deck-lists-magic-online-products-game-info"

In [3]:
# create a new chrome page. Needs chromedriver.exe to be in path. I put next to anaconda stuff
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(base_url)
driver.maximize_window()

In [4]:
#Uses filter to access only certain types of decks

driver.execute_script("window.scrollTo(0, 200)") 
Filterby=driver.find_element_by_id('event_filter')
Filterby.click()
time.sleep(1)
Modern_button=driver.find_element_by_xpath("//li[contains(text(), 'Modern')]")
Modern_button.click()
time.sleep(1)
Search_button=driver.find_element_by_id('custom-search-submit')
Search_button.click()

# select by visible text
# select.select_by_visible_text('Modern')

In [5]:
#After opening the url above, Selenium clicks the scrolls through all the see more archives
'''
scrolls to bottom of page
clicks da fish link to see more info
compares old window height with new one
if equal, continually checks for 60 seconds
if not continues to scroll to the bottom and clicks the link to see more
timeout= time willing to wait for page to load for each click of the see more button
'''

SCROLL_PAUSE_TIME = 1

timeout = 10

while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # touch da fishy
    da_fishy = driver.find_element_by_class_name('see-more-article-listing-section') #See more
    da_fishy.click() #click see more

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    i=0
    while (new_height == last_height) and (i<timeout) :

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(1)
        i+=1
    if i==timeout:
        break



In [6]:
#Handoff to BS and find the aricles we want
soup=BeautifulSoup(driver.page_source,"html.parser")
# Opens all links and gets all data
links=[]
for link in soup.find("div",{"class":"articles-listing"}).find_all('a'):
    a=link.get('href')
    links.append(a)

len(links)

18

In [8]:
# Meat of the scrape
'''
Go to each link and scrape the useful info

'''
Headers=['Deck Name', 'Card Type','Card Count', 'Card Name', 'Card Link', \
         'Sideboard Card Count', 'Sideboard Card Name', 'Sideboard Link']
deck_data_ls=[]
deck_name_ls=[]
for page in links:
    page_url=base_dir+page
    r=requests.get(page_url)
    c=r.content
    page_soup=BeautifulSoup(c,"html.parser")
    
    for deckname in page_soup.find_all("div",{"class":"deck-group"}):
        deck_name=deckname.find("span",{"class":"deck-meta"}).h4.string
        deck_name_ls.append(deck_name)
        card_type=[]
        card_count=[]
        card_name=[]
        card_link=[]
        sideb_count=[]
        sideb_name=[]
        sideb_link=[]
        
        for card in deckname.find("div", {"class":"sorted-by-overview-container"}).find_all("div",{"class":"element"}):
            card_type.append(card.h5.text)
            card_count.append(card.find("span",{"class":"card-count"}).text)
            card_name.append(card.find("span",{"class":"card-name"}).text)
            card_link.append(card.find("span",{"class":"card-name"}).a.get('href'))
            
        for sideb in deckname.find("div", {"class":"sorted-by-sideboard-container"}).find_all("span",{"class":"row"}):
            sideb_count.append(sideb.find("span",{"class":"card-count"}).text)
            sideb_name.append(sideb.find("span",{"class":"card-name"}).text)
            
            try:
                sideb_link.append(sideb.find("span",{"class":"card-name"}).a.get('href'))
            except AttributeError:
                sideb_link.append('NaN')
            
        deck_data=dict(zip(Headers,[deck_name,card_type, card_count,\
                             card_name, card_link, sideb_count, sideb_name, sideb_link])) 
        deck_data_ls.append(deck_data)


In [9]:
Data_temp=pd.DataFrame(columns=Headers)
Data_set=Data_temp.append(deck_data_ls)
Data_set

,Deck Name,Card Type,Card Count,Card Name,Card Link,Sideboard Card Count,Sideboard Card Name,Sideboard Link
0,ninell (4-0),"[Creature (9), Sorcery (5), Instant (24), Land...","[2, 1, 3, 1]","[Gurmag Angler, Rise // Fall, Cryptic Command,...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2]","[Anger of the Gods, Damnation, Dispel, Enginee...",[http://gatherer.wizards.com/Pages/Search/Defa...
1,Ark4n (4-0),"[Planeswalker (3), Creature (8), Sorcery (3), ...","[1, 1, 3, 2, 4]","[Elspeth, Sun's Champion, Glen Elendra Archmag...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[1, 1, 2, 2, 4, 2, 3]","[Negate, Supreme Verdict, Celestial Purge, Dis...",[http://gatherer.wizards.com/Pages/Search/Defa...
2,TonyMontana (3-1),"[Creature (31), Instant (7), Land (22)]","[2, 3, 1]","[Anafenza, Kin-Tree Spirit, Chord of Calling, ...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 1, 1, 1, 3, 1, 1, 1, 1, 3]","[Abrupt Decay, Burrenton Forge-Tender, Eidolon...",[http://gatherer.wizards.com/Pages/Search/Defa...
3,Konsti (3-1),"[Creature (10), Sorcery (5), Instant (23), Lan...","[1, 1, 2, 1]","[Grim Lavamancer, Languish, Cryptic Command, B...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]","[Dispel, Anger of the Gods, Countersquall, Dur...",[http://gatherer.wizards.com/Pages/Search/Defa...
4,FakiVll (3-1),"[Planeswalker (4), Creature (14), Sorcery (9),...","[4, 4, 1, 2, 4]","[Liliana of the Veil, Dark Confidant, Duress, ...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 2, 2, 4, 2, 3]","[Terminate, Thoughtseize, Drown in Sorrow, Ful...",[http://gatherer.wizards.com/Pages/Search/Defa...
5,Navas (3-1),"[Creature (11), Sorcery (7), Instant (15), Enc...","[4, 1, 2, 4, 1]","[Deceiver Exarch, Flame Slash, Cryptic Command...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1]","[Dispel, Roast, Ancient Grudge, Anger of the G...",[http://gatherer.wizards.com/Pages/Search/Defa...
6,gaku0221 (3-1),"[Creature (23), Sorcery (4), Instant (7), Arti...","[1, 4, 4, 2, 2]","[Burning-Tree Shaman, Search for Tomorrow, Sum...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[1, 2, 1, 2, 2, 4, 1, 2]","[Anger of the Gods, Engineered Explosives, Hel...",[http://gatherer.wizards.com/Pages/Search/Defa...
7,NecNick (3-1),"[Planeswalker (5), Creature (14), Sorcery (7),...","[1, 4, 4, 2, 4]","[Chandra, Pyromaster, Dark Confidant, Inquisit...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 2, 1, 1, 3, 1, 1, 1, 2, 1]","[Ancient Grudge, Anger of the Gods, Boil, Dure...",[http://gatherer.wizards.com/Pages/Search/Defa...
8,Teletender (3-1),"[Creature (26), Sorcery (4), Instant (7), Land...","[3, 4, 3, 2]","[Birds of Paradise, Lingering Souls, Abrupt De...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 4, 4, 2, 3]","[Choke, Fulminator Mage, Kor Firewalker, Rest ...",[http://gatherer.wizards.com/Pages/Search/Defa...
9,Kerrick_ (3-1),"[Creature (29), Instant (8), Land (23)]","[4, 4, 1]","[Birds of Paradise, Chord of Calling, Fire-Lit...",[http://gatherer.wizards.com/Pages/Search/Defa...,"[2, 2, 1, 3, 1, 1, 1, 1, 3]","[Fulminator Mage, Avalanche Riders, Eidolon of...",[http://gatherer.wizards.com/Pages/Search/Defa...


In [17]:
Data_set.loc[100,'Card Link']

['http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BDeceiver%5D+%5BExarch%5D',
 'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BRoast%5D',
 'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BCryptic%5D+%5BCommand%5D',
 'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BSplinter%5D+%5BTwin%5D',
 'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BDesolate%5D+%5BLighthouse%5D']